In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os


import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from qkeras.utils import model_save_quantized_weights

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning


from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
import math


from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights, load_qmodel
import hls4ml
import qkeras.utils

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [69]:
from importlib import reload
import estimator as est
reload(est)

<module 'estimator' (<_frozen_importlib_external._NamespaceLoader object at 0x7f3e79075c90>)>

# # Gather all pretrained weights into an array to create a pool of weights and select from when testing the estimator with the generate_sparse function

In [16]:
from testing_utils import generate_sparse
from importlib import reload
reload(testing_utils)

<module 'testing_utils' from '/home/dmitsas/Downloads/dse/testing/estimator_testing/testing_utils.py'>

In [18]:
rf_range = [1,2,3,4,10]


layer_num = 50
models = []
RF = []
precision = []
sparsity = []
selector = []
bits = 8
int_bits = 0
impl = {'LUT':0,'DSP':1}
select = 'DSP'
model_layers = [[] for x in range(layer_num)]
initialiser = tf.keras.initializers.lecun_uniform()
for prec in range(8,1,-1):
    x = 0
    for reuse in rf_range: 
        models = []
        RF = []
        precision = []
        sparsity = []
        selector = []
        filename = "./results_DSP/results/prec_" + str(prec) + "/reuse_" + str(reuse) + "/model_"
        for i in range(x,layer_num,1):  
                model = Sequential()
                layer_1 = random.randint(8,25)
                layer_2 = random.randint(4,60)                           
                layer_3 = random.randint(4,64)                          
                layer_4 = random.randint(8,40)
                model_layers[i].append(layer_1)
                model_layers[i].append(layer_2)
                model_layers[i].append(layer_3)
                model_layers[i].append(layer_4)
                
                model = Sequential()
                model.add(QDense(layer_2, input_shape=(layer_1,), name='fc1', kernel_quantizer=quantized_bits(bits,int_bits,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(bits,int_bits,alpha=1),
                                kernel_initializer=initialiser, kernel_regularizer=l1(0.0001)   ))
                model.add(QActivation(activation=quantized_relu(bits,int_bits,use_stochastic_rounding=False), name='relu1'))
                model.add(QDense(layer_3, name='fc2',
                                kernel_quantizer=quantized_bits(bits,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(bits,int_bits,alpha=1),
                                kernel_initializer=initialiser, kernel_regularizer=l1(0.0001 ) ))
                model.add(QActivation(activation=quantized_relu(bits,int_bits,use_stochastic_rounding=False), name='relu2'))
                model.add(QDense(layer_4, name='fc3',
                                 kernel_quantizer=quantized_bits(bits,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(bits,int_bits,alpha=1),
                                 kernel_initializer=initialiser, kernel_regularizer=l1(0.0001 ) ))
           
                model.add(Activation(activation='softmax', name='softmax'))
                sparsity.append(np.random.uniform(0.2,0.8))
                for layer in range(0,len(model.layers),2):
                    generate_sparse(model,layer,model.weights[layer].shape[0],model.weights[layer].shape[1],sparsity[i])

                models.append(model)
                model.save(filename + str(i) + "/KERAS_check_best_mode.h5")
                #RF.append(random.randint(1,3))
                #RF.append(1)
                selector.append(impl[select])
                RF.append(reuse)
                precision.append(prec)
                
                if not (os.path.exists(filename + str(i))):
                    os.makedirs(filename + str(i))
                with open(filename + str(i) + "/layers.txt",'w') as f:
                    f.write(str(model_layers[i][0]) + " " + str(model_layers[i][1]))
                    f.close
                with open(filename + str(i) + "/precision.txt",'w') as f:
                    f.write(str(precision[i]))
                    f.close

                with open(filename + str(i) + "/reuse.txt",'w') as f:
                    f.write(str(int(RF[i])))
                    f.close



                
                config = hls4ml.utils.config_from_keras_model(models[i],granularity='name')
                #config['Model']['Precision']='fixed<16,4>'
                config['LayerName']['fc1_input']['Precision']['result'] = 'fixed<' + str(precision[i]) + ',2>'
                config['LayerName']['relu1']['Precision']['result'] = 'ap_ufixed<' + str(precision[i]) + ',0,AP_RND_CONV,AP_SAT>'
                config['LayerName']['relu2']['Precision']['result'] = 'ap_ufixed<' + str(precision[i]) + ',0,AP_RND_CONV,AP_SAT>'
                #config['LayerName']['relu3']['Precision']['result'] = 'ap_ufixed<' + str(precision[i]) + ',0,AP_RND_CONV,AP_SAT>'

                #selector.append(random.randint(0,1))
                            #config['LayerName']['output']['Precision']['result'] = 'fixed<16,4>'
                config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
                config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
                config['Model']['ReuseFactor'] = RF[i]
                #config['LayerName']['fc1']['Precision']['result'] = 'fixed<16,4>'
                #config['LayerName']['fc2']['Precision']['result'] = 'fixed<16,4>'
                models[i].save(filename + str(i) + "/KERAS_check_best_mode.h5")
                hls_model = hls4ml.converters.convert_from_keras_model(
                    models[i],hls_config = config,output_dir = filename + str(i) + '/hls4ml_prj',part = 'xc7z007s-clg225-2')
                hls_model.compile()
                os.system('cp -r ./scripts/project_orig.tcl ./scripts/build_prj_orig.tcl ./scripts/init.sh ./scripts/build_prj_orig_lut.tcl ./scripts/init_lut.sh ' + filename + str(i) + '/hls4ml_prj')
                start_dir = os.getcwd()
                os.chdir(filename + str(i) + '/hls4ml_prj')
                if selector[i] == 0:
                    os.system('bash init_lut.sh')
                    os.system('rm init.sh')
                elif selector[i] == 1:
                    os.system('bash init.sh')
                    os.system('rm init_lut.sh')
                #os.chdir('/home/dmitsas/Downloads/notebooks/misc/')
                os.chdir(start_dir)
                if i==49:
                    for k in range(layer_num+1):
                        os.system("rm -r " + filename + str(k) + "/hls4ml_prj/myproject_prj/solution1/.autopilot")
                with open(filename + str(i) + "/impl.txt",'w') as f:
                    f.write(str(selector[i]))
                    f.close

for k in range(layer_num+1):
            os.system("rm -r " + filename + str(k) + "/hls4ml_prj/myproject_prj/solution1/.autopilot")
        
        

In [11]:
for prec in range(8,1,-1):
    for reuse in rf_range:
        filename = "./final_results/results/prec_" + str(prec) + "/reuse_" + str(reuse) + "/model_"
        for k in range(layer_num+1):
            os.system("rm -r " + filename + str(k) + "/hls4ml_prj/myproject_prj/solution1/.autopilot")

rm: cannot remove './final_results/results/prec_8/reuse_1/model_0/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_1/model_1/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_1/model_2/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_1/model_3/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_1/model_4/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_1/model_5/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_1/model_6/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove '.

rm: cannot remove './final_results/results/prec_8/reuse_3/model_12/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_3/model_13/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_3/model_14/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_3/model_15/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_3/model_16/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_3/model_17/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_3/model_18/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot re

rm: cannot remove './final_results/results/prec_8/reuse_10/model_23/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_10/model_24/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_10/model_25/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_10/model_26/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_10/model_27/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_10/model_28/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_8/reuse_10/model_29/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: ca

rm: cannot remove './final_results/results/prec_7/reuse_2/model_33/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_2/model_34/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_2/model_35/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_2/model_36/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_2/model_37/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_2/model_38/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_2/model_39/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot re

rm: cannot remove './final_results/results/prec_7/reuse_4/model_41/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_4/model_42/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_4/model_43/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_4/model_44/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_4/model_45/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_4/model_46/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_7/reuse_4/model_47/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot re

...

rm: cannot remove './final_results/results/prec_3/reuse_1/model_46/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_1/model_47/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_1/model_48/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_1/model_49/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_1/model_50/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_2/model_0/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_2/model_1/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remo

rm: cannot remove './final_results/results/prec_3/reuse_3/model_50/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_4/model_0/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_4/model_1/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_4/model_2/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_4/model_3/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_4/model_4/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_3/reuse_4/model_5/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove '

rm: cannot remove './final_results/results/prec_2/reuse_1/model_3/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_1/model_4/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_1/model_5/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_1/model_6/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_1/model_7/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_1/model_8/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_1/model_9/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove '.

rm: cannot remove './final_results/results/prec_2/reuse_3/model_7/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_3/model_8/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_3/model_9/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_3/model_10/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_3/model_11/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_3/model_12/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_3/model_13/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remov

rm: cannot remove './final_results/results/prec_2/reuse_10/model_9/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_10/model_10/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_10/model_11/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_10/model_12/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_10/model_13/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_10/model_14/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: cannot remove './final_results/results/prec_2/reuse_10/model_15/hls4ml_prj/myproject_prj/solution1/.autopilot': No such file or directory
rm: can